<a href="https://colab.research.google.com/github/sam80FDS/Sam/blob/main/Master.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ==============================================================================
#  HEADLESS STUDIO v31.0 (The "Nuclear" Audio Fix)
#  Status: TESTED.
#  Fix: Deletes 'requirements.txt' to prevent auto-install crashes.
# ==============================================================================

import os
import time
import subprocess

# ------------------------------------------------------------------------------
# STEP 1: SYSTEM PREP
# ------------------------------------------------------------------------------
print("🛠️ STEP 1/6: SYSTEM PREP...")
!rm -rf /content/ComfyUI
!apt-get update -qq
!apt-get install -y ffmpeg libsndfile1 pkg-config -qq

# ------------------------------------------------------------------------------
# STEP 2: INSTALL COMFYUI
# ------------------------------------------------------------------------------
print("🚀 STEP 2/6: INSTALLING COMFYUI...")
%cd /content
!git clone https://github.com/comfyanonymous/ComfyUI
%cd /content/ComfyUI
!pip install -r requirements.txt --quiet

# ------------------------------------------------------------------------------
# STEP 3: INSTALL NODES
# ------------------------------------------------------------------------------
print("📦 STEP 3/6: CLONING NODES...")
%cd /content/ComfyUI/custom_nodes
!git clone https://github.com/ltdrdata/ComfyUI-Manager.git
!git clone https://github.com/Kosinkadink/ComfyUI-VideoHelperSuite.git
!git clone https://github.com/cubiq/ComfyUI_IPAdapter_plus.git
!git clone https://github.com/smthemex/ComfyUI_ParlerTTS.git
!git clone https://github.com/gitmylo/ComfyUI-audio-nodes.git

# ------------------------------------------------------------------------------
# STEP 4: THE MANUAL FIX (Preventing Conflicts)
# ------------------------------------------------------------------------------
print("❤️‍🩹 STEP 4/6: APPLYING AUDIO FIX...")

# A. DELETE THE FILE THAT CAUSES ERRORS
# We delete requirements.txt so ComfyUI doesn't try to run it automatically.
%cd /content/ComfyUI/custom_nodes/ComfyUI-audio-nodes
!rm requirements.txt

# B. MANUAL INSTALL (Safe Mode)
%cd /content
# 1. Install AudioCraft (No Dependencies to save GPU drivers)
!pip install --no-deps git+https://github.com/facebookresearch/audiocraft.git

# 2. Install AudioTools (Needed for Parler)
!pip install git+https://github.com/descriptinc/audiotools.git

# 3. Install missing helpers (Safe list only)
!pip install soundfile librosa encodec funcy omegaconf hydra-core protobuf sentencepiece transformers xformers --quiet

# ------------------------------------------------------------------------------
# STEP 5: DOWNLOAD MODELS
# ------------------------------------------------------------------------------
print("💾 STEP 5/6: DOWNLOADING ASSETS...")
# Visuals
!wget -nc -O /content/ComfyUI/models/checkpoints/dreamshaper_8.safetensors https://huggingface.co/Lykon/DreamShaper/resolve/main/DreamShaper_8_pruned.safetensors
!wget -nc -O /content/ComfyUI/models/checkpoints/svd_xt.safetensors https://huggingface.co/stabilityai/stable-video-diffusion-img2vid-xt/resolve/main/svd_xt.safetensors

# Audio
!wget -nc -O /content/ComfyUI/models/checkpoints/musicgen_small.safetensors https://huggingface.co/facebook/musicgen-small/resolve/main/model.safetensors
!mkdir -p /content/ComfyUI/models/parler_tts
!wget -nc -O /content/ComfyUI/models/parler_tts/parler-tts-mini-v1.safetensors https://huggingface.co/parler-tts/parler-tts-mini-v1/resolve/main/model.safetensors

# Helpers
!mkdir -p /content/ComfyUI/models/upscale_models
!wget -nc -O /content/ComfyUI/models/upscale_models/RealESRGAN_x4plus.pth https://github.com/xinntao/Real-ESRGAN/releases/download/v0.1.0/RealESRGAN_x4plus.pth
!mkdir -p /content/ComfyUI/models/vfi
!wget -nc -O /content/ComfyUI/models/vfi/rife47.pth https://github.com/styler00dollar/VSGAN-tensorrt-docker/releases/download/models/rife47.pth

# ------------------------------------------------------------------------------
# STEP 6: LAUNCH
# ------------------------------------------------------------------------------
print("🌩️ STEP 6/6: LAUNCHING...")
%cd /content/ComfyUI
!wget -nc https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
!dpkg -i cloudflared-linux-amd64.deb

print("----------------------------------------------------------------")
print("✅ READY. Please copy the new trycloudflare link below.")
print("----------------------------------------------------------------")

!python main.py --listen --dont-print-server & cloudflared tunnel --url http://localhost:8188
